In [1]:
from forest.forest_subset_features.class_forest import CustomEnsembleClassifier
import pandas as pd
import time
import os

In [2]:
from sklearn.model_selection import train_test_split

#data_seismic = pd.read_csv("datasets/seismic/seismic_bin.csv")
data_test = pd.read_csv("datasets/example_datasets/stacked.csv")

data = data_test


X = data.drop(columns=['y'])  # All columns except the target
y = data['y']                 # Only the target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y, random_state=42)

stacked_train = pd.concat([y_train, X_train], axis=1, ignore_index=False)
stacked_test = pd.concat([y_test, X_test],axis=1, ignore_index=False)

train_data = stacked_train
test_data = stacked_test

#feature_columns = train_data.columns[1:]

feature_columns = X_train.columns
#print(feature_columns)

In [3]:
tree_kwargs = dict(depth=8, criterion='gini', target_label='y', features=list(range(X_train.shape[1])))

ensemble = CustomEnsembleClassifier(n_estimators=2, tree_kwargs=tree_kwargs, random_state=42, cores_to_use=6)

In [4]:
ensemble.parallel_fit(X_train, y_train)

{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}



/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(i[P])
/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(i[P])
/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions i

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/8bff470d010b44198bef214aa765869a-pulp.mps -sec 1800 -timeMode elapsed -branch -printingOptions all -solution /tmp/8bff470d010b44198bef214aa765869a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 1207 RHS
At line 1221 BOUNDS
At line 1464 ENDATA
Problem MODEL has 13 rows, 242 columns and 484 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1800
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.473138 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0.473138 to -1.79769e+308
Probing was tried 0 times and created

AttributeError: 'CustomTreeWrapper' object has no attribute 'features_org_dataset'

In [ ]:
y_pred = ensemble.predict(X_test)

/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(i[P])
/home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/rolling_lookahead_dt_restructured/rolling_lookahead_dt_pulp/oct/optimal_tree_pulp.py:367: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = np.array(i[P])


In [ ]:
res = pd.DataFrame()
print(y_test)
res['y'] = y_test
res['prediction'] = y_pred
print(res)
unique_values = res['prediction'].unique()
print(unique_values)

94     2
42     1
52     1
29     1
106    2
      ..
27     1
123    3
77     2
109    2
38     1
Name: y, Length: 100, dtype: int64
     y  prediction
94   2           1
42   1           1
52   1           1
29   1           2
106  2           1
..  ..         ...
27   1           1
123  3           3
77   2           2
109  2           2
38   1           1

[100 rows x 2 columns]
[1 2 3]


In [ ]:
import numpy as np
import pandas as pd
import copy
import logging
from typing import Union, List


from sklearn.model_selection import train_test_split

#data_seismic = pd.read_csv("datasets/seismic/seismic_bin.csv")
data_test = pd.read_csv("datasets/example_datasets/stacked.csv")

data = data_test


X = data.drop(columns=['y'])  # All columns except the target
y = data['y']                 # Only the target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, stratify=y, random_state=42)

stacked_train = pd.concat([y_train, X_train], axis=1, ignore_index=False)
stacked_test = pd.concat([y_test, X_test],axis=1, ignore_index=False)

train_data = stacked_train
test_data = stacked_test

df = pd.concat([train_data, test_data])
K = sorted(list(set(df.y))) #K is a sorted list of all unique class labels (target categories) present in the combined training and test data

P = [int(i) for i in list(train_data.loc[:, train_data.columns != 'y'].columns)] # all columns != y; .columns retrieves their names, converts them to list and converts them to integers (assumes that string numbers are column names => preprocessing)

print(P)

features_orig_dataset = len(P)

amount_X_consider = int(np.sqrt(features_orig_dataset))
print(amount_X_consider)

#target = df.iloc[:, [0]] 
target = data[['y']]

#features = df.iloc[:, 1:] 
features = data.loc[:, data.columns != 'y']

selected_feature_cols = np.random.choice(features.columns, size=amount_X_consider, replace=True)
selected_features = features[selected_feature_cols]

data = pd.concat([target, selected_features], axis=1)
print(data)

P = [int(i) for i in list(data.loc[:, data.columns != 'y'].columns)]

P = [i for i in range(len(P))]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
11
     y  10  15  84  83  64  85  78  12  103  47  9
0    1   0   0   0   0   0   0   0   0    0   0  0
1    1   1   0   0   0   0   0   0   0    0   0  0
2    1   0   0   0   0   0   0   0   0    0   0  0
3    1   1   1   0   0   0   0   0   0    0   0  0
4    1   1   0   0   0   0   0   0   0    0   0  0
..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  .. ..
173  3   0   0   0   0   0   0   0   0    0   0  1
174  3   0   0   0   0   0   0   1   0    0   

In [ ]:
for feature in P:
    print(feature)

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
def gini_index(arr: np.array,
               instance_size: int,
               K: list,
               y_idx: int = 0,
               weighted: bool = True):
    """

    :param arr:
    :param instance_size:
    :param K:
    :param y_idx:
    :param weighted:
    :return:
    """
    sum_ = 0
    for k in K:
        sum_ += np.power(len(arr[np.where(arr[:, y_idx] == k)]) / len(arr), 2)
    sum_ = 1 - sum_
    if weighted:
        sum_ = (len(arr) / instance_size) * sum_
    return sum_

In [ ]:
def calculate_gini_old(data: pd.DataFrame,
                   P: list,
                   K: list,
                   nodes: dict) -> dict:
    """

    :param data: pd.DataFrame — The dataset
    :param P: list Indices of features to consider
    :param K: lsit Unique class labels
    :param nodes: dict — Contains: "leaf_nodes": list of leaf node identifiers. leaf_nodes_path": dict mapping each leaf node to a path (list of values for features).
    :return:
    """
    df_arr = np.array(data)
    n = len(data) # total number of instances.
    gini_dict = dict()
    for leaf_ in nodes["leaf_nodes"]:
        temp = dict() # store Gini values for this leaf
        first_var = nodes["leaf_nodes_path"][leaf_][0] #{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
        second_var = nodes["leaf_nodes_path"][leaf_][1] #first_var, second_var — the first two values in the path to this leaf (used as feature values for filtering)
        for feature_i in P:
            arr = df_arr[np.where((df_arr[:, feature_i] == first_var))] #Selects rows where feature feature_i equals first_var.

            for feature_j in P:
                arr_2 = arr[np.where(arr[:, feature_j] == second_var)] #Further filters to rows where feature feature_j equals second_var
                if len(arr_2) > 0: #Calculate Gini index for arr_2 (so for all rows matching (1, 0; having/not having feature i) the decision variables)
                    temp[feature_i, feature_j] = gini_index(arr=arr_2,
                                                            instance_size=n,
                                                            K=K,
                                                            weighted=True)
        gini_dict[leaf_] = copy.deepcopy(temp)
        del temp
        del arr

    return gini_dict

In [ ]:
def generate_nodes(depth: int) -> list:
    """

    :param depth:
    :return:
    """
    nodes = list(range(1, int(round(2 ** (depth + 1)))))
    parent_nodes = nodes[0: 2 ** (depth + 1) - 2 ** depth - 1]
    leaf_nodes = nodes[-2 ** depth:]
    return parent_nodes, leaf_nodes

In [ ]:
leaf_nodes_path = {4: [1, 1], #dict for all leafes in depth 2 tree and der respective split conditions
                       5: [1, 0],
                       6: [0, 1],
                       7: [0, 0]}
depth = 2
parent_nodes, leaf_nodes = generate_nodes(depth) # (for depth 2) returns [1,2,3] and [4,5,6,7]

nodes = dict()
nodes["leaf_nodes"] = leaf_nodes
nodes["leaf_nodes_path"] = leaf_nodes_path


In [ ]:
coef_dict = calculate_gini_old(data=data,
                                    P=P,
                                    K=K,
                                    nodes=nodes)

print(coef_dict)

{4: {(0, 0): np.float64(0.0), (0, 1): np.float64(0.0), (0, 2): np.float64(0.0), (0, 3): np.float64(0.0), (0, 4): np.float64(0.0), (0, 6): np.float64(0.0), (0, 7): np.float64(0.0), (0, 8): np.float64(0.0), (0, 9): np.float64(0.0), (0, 10): np.float64(0.0), (1, 0): np.float64(0.0), (1, 1): np.float64(0.02775941837409121), (1, 2): np.float64(0.0), (1, 3): np.float64(0.0), (1, 4): np.float64(0.0056179775280898875), (1, 7): np.float64(0.0), (1, 8): np.float64(0.0), (2, 0): np.float64(0.0), (2, 1): np.float64(0.0), (2, 2): np.float64(0.026484751203852328), (2, 4): np.float64(0.0), (2, 5): np.float64(0.0), (2, 6): np.float64(0.0), (2, 7): np.float64(0.0), (3, 0): np.float64(0.0), (3, 1): np.float64(0.0), (3, 3): np.float64(0.059925093632958795), (3, 5): np.float64(0.0), (3, 7): np.float64(0.0), (3, 8): np.float64(0.014044943820224719), (3, 9): np.float64(0.0), (3, 10): np.float64(0.0), (4, 0): np.float64(0.0), (4, 1): np.float64(0.0056179775280898875), (4, 2): np.float64(0.0), (4, 4): np.floa